In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageFilter
import pytesseract
from scipy.ndimage import *
import scipy.signal as scisignal
import skimage.exposure as expo
import skimage.morphology as morpha
from skimage.measure import *
import scipy.ndimage as ndimg

In [68]:
aspect_ratio = lambda image: image.size[0] / image.size[1]

In [69]:
# Import image
image = Image.open("carplate1.jpg")

# Resize the image while maintaining aspect ratio
new_size_of_image = (400,round(400/aspect_ratio(image)))
new_image = image.resize(new_size_of_image)

# Convert Image to Grayscale (Colour Image Processing)
image_grayscale = new_image.convert("L")

# Perform median filtering on the image
median_filtered_image = image_grayscale.filter(ImageFilter.MedianFilter(3))

# Structural element for image dilation and erosion
structural_element = np.array([[0,1,0],
                      [1,1,1],
                      [0,1,0]])

# Converting median filtered image to its binary form for image dilation and erosion
binary_image = median_filtered_image.convert("1")

# Performing Image Dilation using the structural element
dilated_image = np.int64(ndimg.binary_dilation(input=binary_image,structure=structural_element))

# Performing Image Erosion using the structural element
eroded_image = np.int64(ndimg.binary_erosion(input=binary_image,structure=structural_element))

# Morphological Element for enhancing edges of the image
gradient_image = dilated_image - eroded_image

# Scaling the image to [0,1] to convert the image to double precision
gradient_image_scaled = gradient_image / gradient_image.max()

# Convolution of double precision image for beightening the edges
kernel = [[1,1],
          [1,1]]
convolved_image = scisignal.convolve2d(in1=gradient_image_scaled,in2=kernel,mode="valid")

# Intensity scaling between the range 0 to 1
low_in = 0.5
high_in = 0.7
low_out = 0
high_out = 1
gamma = 0.1
gain = (high_out - low_out) / (high_in - low_in)
intensity_scaled_image = expo.adjust_gamma(image=convolved_image,gamma=gamma,gain=gain)

# Conversion of double image back to binary image
back_binary_image = np.round(intensity_scaled_image / intensity_scaled_image.max())

In [14]:
back_binary_image

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [15]:
# Elimination of edges of the licence plate

eroded_back_binary_image = np.float64(ndimg.binary_erosion(input=back_binary_image,structure=np.ones(shape=(1,50))))
eliminated_edges_image = back_binary_image - eroded_back_binary_image

# Filling all the regions of the image with holes
eliminated_edges_image_inverted = 1 - eliminated_edges_image
filled_image_with_holes_before_invert = ndimg.binary_fill_holes(eliminated_edges_image_inverted)
filled_image_with_holes = 1 - filled_image_with_holes_before_invert

# Thinning the image filled with holes to make sure that characters do not match with each other (THIS STEP IS THE ACTUAL STEP FOR MORPHOLOGICAL PROCESSING
thinned_image = np.float64(morpha.thin(filled_image_with_holes))
eroded_thinned_image = np.float64(ndimg.binary_erosion(input=thinned_image,structure=np.ones(shape=(3,1))))

In [16]:
# Selecting all the pixels of area more than 100 (Image masking)
image_with_labels, number_of_labels = ndimg.label(eroded_thinned_image)
area_of_each_component = np.bincount(image_with_labels.ravel())
mask_with_pixel_areas_greater_than_100 = area_of_each_component >= 100
image_with_pixel_areas_greater_than_100 = np.float64(mask_with_pixel_areas_greater_than_100[image_with_labels])
final_image_for_vehicle_number_detection = np.int64(image_with_pixel_areas_greater_than_100)

In [17]:
# Considering two properties of image regions: 1) "Bounding Boxes" and 2) "Binary Images" corresponding to Bounding Images
image_region_properties = regionprops(final_image_for_vehicle_number_detection)

# Considering the bounding boxes in a matrix of order <Number of Bounding Boxes> X 4;
bounding_boxes = np.vstack([i.bbox for i in image_region_properties])

### Create Templates

In [41]:
def create_templates():
    path_full = "Character Images\\"
    
    A = np.array(Image.open(path_full+'A.bmp'))
    B = np.array(Image.open(path_full+'B.bmp'))
    C = np.array(Image.open(path_full+'C.bmp'))
    D = np.array(Image.open(path_full+'D.bmp'))
    E = np.array(Image.open(path_full+'E.bmp'))
    F = np.array(Image.open(path_full+'F.bmp'))
    G = np.array(Image.open(path_full+'G.bmp'))
    H = np.array(Image.open(path_full+'H.bmp'))
    I = np.array(Image.open(path_full+'I.bmp'))
    J = np.array(Image.open(path_full+'J.bmp'))
    K = np.array(Image.open(path_full+'K.bmp'))
    L = np.array(Image.open(path_full+'L.bmp'))
    M = np.array(Image.open(path_full+'M.bmp'))
    N = np.array(Image.open(path_full+'N.bmp'))
    O = np.array(Image.open(path_full+'O.bmp'))
    P = np.array(Image.open(path_full+'P.bmp'))
    Q = np.array(Image.open(path_full+'Q.bmp'))
    R = np.array(Image.open(path_full+'R.bmp'))
    S = np.array(Image.open(path_full+'S.bmp'))
    T = np.array(Image.open(path_full+'T.bmp'))
    U = np.array(Image.open(path_full+'U.bmp'))
    V = np.array(Image.open(path_full+'V.bmp'))
    W = np.array(Image.open(path_full+'W.bmp'))
    X = np.array(Image.open(path_full+'X.bmp'))
    Y = np.array(Image.open(path_full+'Y.bmp'))
    Z = np.array(Image.open(path_full+'Z.bmp'))
    Afill = np.array(Image.open(path_full+'fillA.bmp'))
    Bfill = np.array(Image.open(path_full+'fillB.bmp'))
    Dfill = np.array(Image.open(path_full+'fillD.bmp'))
    Ofill = np.array(Image.open(path_full+'fillO.bmp'))
    Pfill = np.array(Image.open(path_full+'fillP.bmp'))
    Qfill = np.array(Image.open(path_full+'fillQ.bmp'))
    Rfill = np.array(Image.open(path_full+'fillR.bmp'))
    
    # Number
    one = np.array(Image.open(path_full+'1.bmp'))
    two = np.array(Image.open(path_full+'2.bmp'))
    three = np.array(Image.open(path_full+'3.bmp'))
    four = np.array(Image.open(path_full+'4.bmp'))
    five = np.array(Image.open(path_full+'5.bmp'))
    six = np.array(Image.open(path_full+'6.bmp'))
    seven = np.array(Image.open(path_full+'7.bmp'))
    eight = np.array(Image.open(path_full+'8.bmp'))
    nine = np.array(Image.open(path_full+'9.bmp'))
    zero = np.array(Image.open(path_full+'0.bmp'))
    zerofill = np.array(Image.open(path_full+'fill0.bmp'))
    fourfill = np.array(Image.open(path_full+'fill4.bmp'))
    sixfill = np.array(Image.open(path_full+'fill6.bmp'))
    sixfill2 = np.array(Image.open(path_full+'fill6_2.bmp'))
    eightfill = np.array(Image.open(path_full+'fill8.bmp'))
    ninefill = np.array(Image.open(path_full+'fill9.bmp'))
    ninefill2 = np.array(Image.open(path_full+'fill9_2.bmp'))
    
    # Concatenate images
    letter = np.concatenate([A, Afill, B, Bfill, C, D, Dfill, E, F, G, H, I, J, K, L, M,
                             N, O, Ofill, P, Pfill, Q, Qfill, R, Rfill, S, T, U, V, W, X, Y, Z], axis=1)
    
    number = np.concatenate([one, two, three, four, fourfill, five, six, sixfill, sixfill2,
                             seven, eight, eightfill, nine, ninefill, ninefill2, zero, zerofill], axis=1)
    
    character = np.concatenate([letter, number], axis=1)
    
    return character


### Read Letter

In [42]:

def read_letter(snap):
    # Load the templates of characters into memory
    templates_data = create_templates()
    templates = templates_data['character']

    # Resize the input image
    snap = np.array(Image.fromarray(snap).resize((24, 42)))

    comp = []

    # Correlation of the input image with every image in the template for best matching
    for template in templates:
        sem = np.corrcoef(template.flatten(), snap.flatten())[0, 1]
        comp.append(sem)

    vd = np.argmax(comp) + 1  # Find the index which corresponds to the highest matched character

    # According to the index, assign to 'letter'
    # Alphabets listings
    if vd in [1, 2]:
        letter = 'A'
    elif vd in [3, 4]:
        letter = 'B'
    elif vd == 5:
        letter = 'C'
    elif vd in [6, 7]:
        letter = 'D'
    elif vd == 8:
        letter = 'E'
    elif vd == 9:
        letter = 'F'
    elif vd == 10:
        letter = 'G'
    elif vd == 11:
        letter = 'H'
    elif vd == 12:
        letter = 'I'
    elif vd == 13:
        letter = 'J'
    elif vd == 14:
        letter = 'K'
    elif vd == 15:
        letter = 'L'
    elif vd == 16:
        letter = 'M'
    elif vd == 17:
        letter = 'N'
    elif vd in [18, 19]:
        letter = 'O'
    elif vd in [20, 21]:
        letter = 'P'
    elif vd in [22, 23]:
        letter = 'Q'
    elif vd in [24, 25]:
        letter = 'R'
    elif vd == 26:
        letter = 'S'
    elif vd == 27:
        letter = 'T'
    elif vd == 28:
        letter = 'U'
    elif vd == 29:
        letter = 'V'
    elif vd == 30:
        letter = 'W'
    elif vd == 31:
        letter = 'X'
    elif vd == 32:
        letter = 'Y'
    elif vd == 33:
        letter = 'Z'
    # Numerals listings
    elif vd == 34:
        letter = '1'
    elif vd == 35:
        letter = '2'
    elif vd == 36:
        letter = '3'
    elif vd in [37, 38]:
        letter = '4'
    elif vd == 39:
        letter = '5'
    elif vd in [40, 41, 42]:
        letter = '6'
    elif vd == 43:
        letter = '7'
    elif vd in [44, 45]:
        letter = '8'
    elif vd in [46, 47, 48]:
        letter = '9'
    else:
        letter = '0'

    return letter

### Take Boxes

In [43]:
import numpy as np

def take_boxes(NR, container, chk):
    # Initialize the variable to an empty matrix
    takethisbox = np.empty((0, NR.shape[1]))

    # If bounding box is among the container plus tolerance, take that box and concatenate along the first dimension
    for i in range(NR.shape[0]):
        if container[0] <= NR[i, (2 * chk)] <= container[1]:
            takethisbox = np.vstack((takethisbox, NR[i, :]))

    r = []

    # Finding the indices of the interested boxes among NR since x-coordinate of the boxes will be unique.
    for k in range(takethisbox.shape[0]):
        var = np.where(takethisbox[k, 0] == NR[:, 0])[0]

        # In case if x-coordinate is not unique, then check which box falls under container condition
        if len(var) == 1:
            r.extend(var)
        else:
            M = [NR[var[v], (2 * chk)] >= container[0] and NR[var[v], (2 * chk)] <= container[1] for v in range(len(var))]
            var = [var[v] for v in range(len(var)) if M[v]]
            r.extend(var)

    return r

### Guess The Six

In [62]:
def guess_the_six(Q, W, bsize):
    for l in range(5, 1, -1):
        val = np.where(Q == l)[0]
        var = len(val)

        if var == 0 or var == 1:
            index = (val[0] + 1 if val == 1 else val[0]) if val.size > 0 else 0

            if index == len(Q):
                index = None

            if Q[index] + Q[index + 1] == 6:
                container = [W[index] - (bsize / 2), W[index + 1] + (bsize / 2)]
                break
            elif Q[index] + Q[index - 1] == 6:
                container = [W[index - 1] - (bsize / 2), W[index] + (bsize / 2)]
                break
        else:
            for k in range(var):
                index = val[k] + 1 if val[k] == 1 else val[k]

                if index == len(Q):
                    index = None

                if Q[index] + Q[index + 1] == 6:
                    container = [W[index] - (bsize / 2), W[index + 1] + (bsize / 2)]
                    break
                elif Q[index] + Q[index - 1] == 6:
                    container = [W[index - 1] - (bsize / 2), W[index] + (bsize / 2)]
                    break

            if k != var - 1:
                break

    if l == 2:
        container = []

    return container


### Controlling

In [63]:
import numpy as np

def controlling(NR):
    Q, W = np.histogram(NR[:, 3], bins='auto')  # Histogram of the y-dimension widths of all boxes.
    ind = np.where(Q == 6)[0]  # Find indices from Q corresponding to frequency '6'.

    C_5 = NR[:, 1] * NR[:, 3]  # Taking the advantage of uniqueness of y-coordinate and y-width.
    NR2 = np.column_stack((NR, C_5))  # Appending a new column in NR.

    E, R = np.histogram(NR2[:, 4], bins=20)
    Y = np.where(E == 6)[0]  # Searching for six characters.

    if len(ind) == 1:  # If six boxes of interest are successfully found, record the midpoint of the corresponding bin.
        MP = W[ind]
        binsize = W[1] - W[0]  # Calculate the container size.
        container = [MP - (binsize / 2), MP + (binsize / 2)]  # Calculate the complete container size.
        r = take_boxes(NR, container, 2)
    elif len(Y) == 1:
        MP = R[Y]
        binsize = R[1] - R[0]
        container = [MP - (binsize / 2), MP + (binsize / 2)]  # Calculate the complete container size.
        r = take_boxes(NR2, container, 2.5)  # Call to function take_boxes.
    elif len(ind) == 0 or len(ind) > 1:  # If there is no value of '6' in the Q vector.
        A, B = np.histogram(NR[:, 1], bins=20)  # Use y-coordinate approach only.
        ind2 = np.where(A == 6)[0]
        if len(ind2) == 1:
            MP = B[ind2]
            binsize = B[1] - B[0]
            container = [MP - (binsize / 2), MP + (binsize / 2)]  # Calculate the complete container size.
            r = take_boxes(NR, container, 1)
        else:
            container = guess_the_six(A, B, (B[1] - B[0]))  # Call of function guess_the_six.
            if container:  # If guess_the_six works successfully.
                r = take_boxes(NR, container, 1)  # Call the function take_boxes.
            else:
                container2 = guess_the_six(E, R, (R[1] - R[0]))
                if container2:
                    r = take_boxes(NR2, container2, 2.5)
                else:
                    r = []  # Otherwise assign an empty matrix to 'r'.

    return r


In [64]:
cc = create_templates()

In [65]:
# Retrieving an array comprising the indices for bounding boxes needed for character extraction
character_extraction_bounding_box_indices = controlling(bounding_boxes)

In [66]:

# Assuming 'r' contains the indices of desired boxes
if character_extraction_bounding_box_indices:
    I = [image_region_properties[i]['Image'] for i in character_extraction_bounding_box_indices]  # List of 'Image' property for corresponding indices in 'r'
    number_on_plate = ""  # Initializing the variable for the number plate string

    for v, index in enumerate(character_extraction_bounding_box_indices, start=1):
        N = image_region_properties[index]['Image']  # Extracting the binary image corresponding to the index in 'r'
        letter = read_letter(N)  # Reading the letter corresponding to the binary image 'N'

        # Since it wouldn't be easy to distinguish between '0' and 'O',
        # using the characteristic of plates in Karachi to decide whether it is '0' or 'O'
        while letter == 'O' or letter == '0':
            if v <= 3:
                letter = 'O'
            else:
                letter = '0'
            break

        number_on_plate += letter  # Appending every subsequent character in the 'no_plate' variable

    with open('number_on_plate.txt', 'w') as file:
        file.write(number_on_plate)  # Writing the number plate to the text file

    # Uncomment the portion of code below if a database is to be organized
    # load the database ('DB') from a .mat file
    # for record in DB:
    #     record_plate = record['PlateNumber']
    #     if no_plate == record_plate:
    #         print(record)
    #         print('*-*-*-*-*-*-*')

    # Open the text file with the number plate written
    with open('noPlate.txt', 'r') as file:
        print(file.read())
else:
    print('Unable to extract the characters from the number plate.')
    print('The characters on the number plate might not be clear or touching with each other or boundaries.')


Unable to extract the characters from the number plate.
The characters on the number plate might not be clear or touching with each other or boundaries.
